# TODO Document

In [1]:
# CHANGE THIS PARAMETER
recommendee = "taapaye"

In [2]:
import functools
import os
import pickle

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf
from tqdm import tqdm


@functools.wraps(smf.ols)
def lm(*args, **kwargs):
    return smf.ols(*args, **kwargs).fit()

In [3]:
outdir = f"../../data/recommendations/{recommendee}"
os.chdir(outdir)

In [4]:
anime = pd.read_csv("../../cleaned_data/anime.csv")

In [5]:
user_df = pickle.load(open("user_anime_list.pkl", "rb"))

In [6]:
records = []
for signal in tqdm(['strictrelations', 'weakrelations', 'allrelations']):
    related_df = pickle.load(open(f"{signal}_loocv.pkl", "rb"))
    pred_df = user_df.merge(related_df, on="anime_id", how="left").fillna(0)
    model = lm("score ~ delta", pred_df)
    rmse = np.sqrt(((pred_df['score'] - model.predict(pred_df)) ** 2).mean())
    records.append((signal, model.rsquared_adj, rmse))
parameters = pd.DataFrame.from_records(records, columns = ['signal', 'rsquared', 'rmse'])

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 55.67it/s]


In [7]:
parameters.sort_values(by='rsquared', ascending=False)

,signal,rsquared,rmse
1,weakrelations,0.081794,1.208098
0,strictrelations,0.069987,1.215840
2,allrelations,0.060529,1.222007


In [8]:
outdir = "parameters"
if not os.path.exists(outdir):
    os.mkdir(outdir)
os.chdir(outdir)

In [9]:
parameters.to_pickle("related.all.pkl")
parameters.sort_values(by='rsquared', ascending=False).head(1).to_pickle("related.best.pkl")

In [10]:
best = parameters.sort_values(by='rsquared', ascending=False).head(1)['signal'].squeeze()
pickle.load(open(f"../{best}_loocv.pkl", "rb")).to_pickle('../related_loocv.pkl')
pickle.load(open(f"../{best}.pkl", "rb")).to_pickle('../related.pkl')

### 